## Setup

In [1]:
import logging

from dotenv import load_dotenv

from autogpt.logs.config import configure_logging
from autogpt.core.resource.model_providers import MultiProvider

load_dotenv()
configure_logging(debug=True)

llm = MultiProvider()
logger = logging.getLogger()

## Prompt

In [2]:
import json
import logging
import re

from pydantic import BaseModel, Field

from autogpt.config.ai_directives import AIDirectives
from autogpt.config.ai_profile import AIProfile
from autogpt.core.prompting import PromptStrategy
from autogpt.core.prompting.schema import ChatPrompt, LanguageModelClassification
from autogpt.core.resource.model_providers import ChatMessage
from autogpt.core.resource.model_providers.schema import AssistantChatMessage, CompletionModelFunction
from autogpt.core.utils.json_schema import JSONSchema
from autogpt.core.utils.json_utils import extract_dict_from_json
from autogpt.models.utils import ModelWithSummary
from autogpt.prompts.utils import format_numbered_list
from autogpt.utils.exceptions import InvalidAgentResponseError

_RESPONSE_INTERFACE_NAME = "AssistantResponse"


class AssistantThoughts(ModelWithSummary):
    observations: str = Field(
        ..., description="Relevant observations from your last action (if any)"
    )
    self_criticism: str = Field(..., description="Constructive self-criticism")
    reasoning: str = Field(..., description="Reasoning towards a new plan")
    plan: list[str] = Field(
        ..., description="Short list that conveys your long-term plan. Parallelize where possible."
    )

    def summary(self) -> str:
        return self.reasoning


class OneShotFlowAgentActionProposal(BaseModel):
    thoughts: AssistantThoughts
    python_code: str = Field(
        ..., description=(
            "Write Python code to execute your plan as efficiently as possible. "
            "Try to do as much as possible without making any uninformed guesses. "
            "Use ONLY the listed available functions and built-in Python features. "
            "Leverage the given magic functions to implement function calls for which the "
            "arguments can't be determined yet."
        )
    )

FINAL_INSTRUCTION: str = (
    # "Determine exactly one command to use next based on the given goals "
    # "and the progress you have made so far, "
    # "and respond using the JSON schema specified previously:"
    "Write Python code to execute your plan as efficiently as possible. "
    "Your code will be executed directly without any editing: "
    "if it doesn't work you will be held responsible. "
    "Use ONLY the listed available functions and built-in Python features. "
    "Do not make uninformed assumptions (e.g. about the content or format of an unknown file). "
    "Leverage the given magic functions to implement function calls for which the "
    "arguments can't be determined yet. Reduce the amount of unnecessary data passed into "
    "these magic functions where possible, because magic costs money and magically "
    "processing large amounts of data is expensive."
)


class OneShotFlowAgentPromptStrategy(PromptStrategy):
    def __init__(self):
        self.response_schema = JSONSchema.from_dict(OneShotFlowAgentActionProposal.schema())
        self.logger = logging.getLogger(self.__class__.__name__)

    @property
    def model_classification(self) -> LanguageModelClassification:
        return LanguageModelClassification.FAST_MODEL  # FIXME: dynamic switching

    def build_prompt(
        self,
        *,
        messages: list[ChatMessage],
        task: str,
        ai_profile: AIProfile,
        ai_directives: AIDirectives,
        functions: list[CompletionModelFunction],
        **extras,
    ) -> ChatPrompt:
        """Constructs and returns a prompt with the following structure:
        1. System prompt
        3. `cycle_instruction`
        """
        system_prompt, response_prefill = self.build_system_prompt(
            ai_profile=ai_profile,
            ai_directives=ai_directives,
            functions=functions,
        )

        final_instruction_msg = ChatMessage.user(FINAL_INSTRUCTION)

        return ChatPrompt(
            messages=[
                ChatMessage.system(system_prompt),
                ChatMessage.user(f'"""{task}"""'),
                *messages,
                final_instruction_msg,
            ],
            prefill_response=response_prefill,
        )

    def build_system_prompt(
        self,
        ai_profile: AIProfile,
        ai_directives: AIDirectives,
        functions: list[CompletionModelFunction],
    ) -> tuple[str, str]:
        """
        Builds the system prompt.

        Returns:
            str: The system prompt body
            str: The desired start for the LLM's response; used to steer the output
        """
        response_fmt_instruction, response_prefill = self.response_format_instruction()
        system_prompt_parts = (
            self._generate_intro_prompt(ai_profile)
            + [
                "## Your Task\n"
                "The user will specify a task for you to execute, in triple quotes,"
                " in the next message. Your job is to complete the task, "
                "and terminate when your task is done."
            ]
            + [
                "## Available Functions\n"
                + self._generate_function_headers(functions)
            ]
            + ["## RESPONSE FORMAT\n" + response_fmt_instruction]
        )

        # Join non-empty parts together into paragraph format
        return (
            "\n\n".join(filter(None, system_prompt_parts)).strip("\n"),
            response_prefill,
        )

    def response_format_instruction(self) -> tuple[str, str]:
        response_schema = self.response_schema.copy(deep=True)

        # Unindent for performance
        response_format = re.sub(
            r"\n\s+",
            "\n",
            response_schema.to_typescript_object_interface(_RESPONSE_INTERFACE_NAME),
        )
        response_prefill = f'{{\n    "{list(response_schema.properties.keys())[0]}":'

        return (
            (
                f"YOU MUST ALWAYS RESPOND WITH A JSON OBJECT OF THE FOLLOWING TYPE:\n"
                f"{response_format}"
            ),
            response_prefill,
        )

    def _generate_intro_prompt(self, ai_profile: AIProfile) -> list[str]:
        """Generates the introduction part of the prompt.

        Returns:
            list[str]: A list of strings forming the introduction part of the prompt.
        """
        return [
            f"You are {ai_profile.ai_name}, {ai_profile.ai_role.rstrip('.')}.",
            # "Your decisions must always be made independently without seeking "
            # "user assistance. Play to your strengths as an LLM and pursue "
            # "simple strategies with no legal complications.",
        ]

    def _generate_function_headers(self, funcs: list[CompletionModelFunction]) -> str:
        return "\n\n".join(f.fmt_header() for f in funcs)

    def parse_response_content(
        self,
        response: AssistantChatMessage,
    ) -> OneShotFlowAgentActionProposal:
        if not response.content:
            raise InvalidAgentResponseError("Assistant response has no text content")

        self.logger.debug(
            "LLM response content:"
            + (
                f"\n{response.content}"
                if "\n" in response.content
                else f" '{response.content}'"
            )
        )
        assistant_reply_dict = extract_dict_from_json(response.content)
        self.logger.debug(
            "Parsing object extracted from LLM response:\n"
            f"{json.dumps(assistant_reply_dict, indent=4)}"
        )

        parsed_response = OneShotFlowAgentActionProposal.parse_obj(assistant_reply_dict)
        if not parsed_response.python_code:
            raise ValueError("python_code is empty")
        return parsed_response

2024-05-10 19:14:26,312 DEBUG cmd.py:1057  Popen(['git', 'version'], cwd=/home/reinier/code/agpt/Auto-GPT/autogpts/autogpt/autogpt/agents/prompt_strategies, stdin=None, shell=False, universal_newlines=False)
2024-05-10 19:14:26,320 DEBUG cmd.py:1057  Popen(['git', 'version'], cwd=/home/reinier/code/agpt/Auto-GPT/autogpts/autogpt/autogpt/agents/prompt_strategies, stdin=None, shell=False, universal_newlines=False)


In [6]:
tools = [
    CompletionModelFunction(
        name="web_search",
        description="Searches the web",
        parameters={
            "query": JSONSchema(
                type=JSONSchema.Type.STRING,
                description="The search query",
                required=True,
            ),
            "num_results": JSONSchema(
                type=JSONSchema.Type.INTEGER,
                description="The number of results to return",
                minimum=1,
                maximum=10,
                required=False,
            ),
        },
    ),
    CompletionModelFunction(
        name="read_webpage",
        description=(
            "Read a webpage, and extract specific information from it."
            " You must specify either topics_of_interest,"
            " a question, or get_raw_content."
        ),
        parameters={
            "url": JSONSchema(
                type=JSONSchema.Type.STRING,
                description="The URL to visit",
                required=True,
            ),
            "topics_of_interest": JSONSchema(
                type=JSONSchema.Type.ARRAY,
                items=JSONSchema(type=JSONSchema.Type.STRING),
                description=(
                    "A list of topics about which you want to extract information "
                    "from the page."
                ),
                required=False,
            ),
            "question": JSONSchema(
                type=JSONSchema.Type.STRING,
                description=(
                    "A question you want to answer using the content of the webpage."
                ),
                required=False,
            ),
            "get_raw_content": JSONSchema(
                type=JSONSchema.Type.BOOLEAN,
                description=(
                    "If true, the unprocessed content of the webpage will be returned. "
                    "This consumes a lot of tokens, so use it with caution."
                ),
                required=False,
            ),
        },
    ),
    CompletionModelFunction(
        name="read_file",
        description="Read a file and return the contents",
        parameters={
            "filename": JSONSchema(
                type=JSONSchema.Type.STRING,
                description="The path of the file to read",
                required=True,
            )
        },
    ),
    CompletionModelFunction(
        name="write_file",
        description="Write a file, creating it if necessary. If the file exists, it is overwritten.",
        parameters={
            "filename": JSONSchema(
                type=JSONSchema.Type.STRING,
                description="The name of the file to write to",
                required=True,
            ),
            "contents": JSONSchema(
                type=JSONSchema.Type.STRING,
                description="The contents to write to the file",
                required=True,
            ),
        },
    ),
    CompletionModelFunction(
        name="list_folder",
        description="List files in a folder recursively",
        parameters={
            "folder": JSONSchema(
                type=JSONSchema.Type.STRING,
                description="The folder to list files in",
                required=True,
            )
        },
    ),
    CompletionModelFunction(
        name="ask_user",
        description="If you need more details or information regarding the given task, "
        "you can ask the user for input.",
        parameters={
            "question": JSONSchema(
                type=JSONSchema.Type.STRING,
                description="The question or prompt to the user",
                required=True,
            )
        },
    ),
    CompletionModelFunction(
        name="extract_value_with_ai",
        description="Magic function to extract information from a body with arbitrary format "
        "into a variable of the specified type.",
        parameters={
            "input": JSONSchema(
                type=JSONSchema.Type.STRING,
                required=True,
            ),
            "instruction": JSONSchema(
                type=JSONSchema.Type.STRING,
                description="Instruction on what information to extract and how to format the output",
                required=True,
            ),
            "output_type": JSONSchema(
                type=JSONSchema.Type.TYPE,
                default=str,
            ),
        },
    ),
    CompletionModelFunction(
        name="extract_information",
        description="Extract information from a document",
        parameters={
            "document_content": JSONSchema(
                type=JSONSchema.Type.STRING,
                required=True,
            ),
            "topics_of_interest": JSONSchema(
                type=JSONSchema.Type.ARRAY,
                items=JSONSchema(type=JSONSchema.Type.STRING),
                required=True,
            ),
        },
    ),
    CompletionModelFunction(
        name="implement_function_arguments",
        description=(
            "Magic function which, IF provided enough information, "
            "will generate the keyword arguments necessary for a specific function."
        ),
        parameters={
            "target_function": JSONSchema(
                type=JSONSchema.Type.STRING,
                description="The function for which to generate arguments",
            ),
            "**kwargs": JSONSchema(
                description=(
                    "Any keyword arguments containing information that should be used "
                    "to determine the arguments for target_function."
                )
            ),
        },
    )
]

In [8]:
from autogpt.core.resource.model_providers.anthropic import AnthropicModelName
from autogpt.core.resource.model_providers.openai import OpenAIModelName
from autogpt.core.runner.client_lib.logging.helpers import dump_prompt

prompter = OneShotFlowAgentPromptStrategy()

prompt = prompter.build_prompt(
    task="Figure out from file1.csv and file2.csv how much was spent on utilities",
    messages=[
        ChatMessage.system(
            "## Context\n"
            '<FolderContextItem '
            'description="The contents of the folder \'.\' in the workspace" '
            'src="." index="1">\n'
            "<file>file1.csv</file>\n"
            "<file>file2.csv</file>\n"
            "</FolderContextItem>"
            "\n\n"
            "When a context item is no longer needed and you are not done yet, "
            "you can hide the item by specifying its index in the list above "
            f"to `close_context_item`.",
        ),
    ],
    ai_profile=AIProfile(),
    ai_directives=AIDirectives(),
    functions=tools,
)
print(dump_prompt(prompt))

result = await llm.create_chat_completion(
    # model_name=AnthropicModelName.CLAUDE3_OPUS_v1,
    model_name=OpenAIModelName.GPT4_TURBO,
    model_prompt=prompt.messages,
    prefill_response=prompt.prefill_response,
    completion_parser=prompter.parse_response_content,
)

print(result.parsed_result.python_code)


============== list ==============
Length: 4 messages
----------------- SYSTEM -----------------
You are AutoGPT, a seasoned digital assistant: capable, intelligent, considerate and assertive. You have extensive research and development skills, and you don't shy away from writing some code to solve a problem. You are pragmatic and make the most out of the tools available to you.

## Your Task
The user will specify a task for you to execute, in triple quotes, in the next message. Your job is to complete the task, and terminate when your task is done.

## Available Functions
def web_search(query: str, num_results: int = None):
    """
    Searches the web
    
    Params:
        query: The search query
        num_results: The number of results to return
    """
    ...

def read_webpage(url: str, topics_of_interest: list[str] = None, question: str = None, get_raw_content: bool = None):
    """
    Read a webpage, and extract specific information from it. You must specify either topics

2024-05-10 20:02:15,670 DEBUG _trace.py:85  connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x7f4c4b5ff210>
2024-05-10 20:02:15,674 DEBUG _trace.py:85  start_tls.started ssl_context=<ssl.SSLContext object at 0x7f4c4b1ebf50> server_hostname='api.openai.com' timeout=5.0
2024-05-10 20:02:15,867 DEBUG _trace.py:85  start_tls.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x7f4c4a7f3490>
2024-05-10 20:02:15,870 DEBUG _trace.py:85  send_request_headers.started request=<Request [b'POST']>
2024-05-10 20:02:15,874 DEBUG _trace.py:85  send_request_headers.complete
2024-05-10 20:02:15,876 DEBUG _trace.py:85  send_request_body.started request=<Request [b'POST']>
2024-05-10 20:02:15,880 DEBUG _trace.py:85  send_request_body.complete
2024-05-10 20:02:15,882 DEBUG _trace.py:85  receive_response_headers.started request=<Request [b'POST']>
2024-05-10 20:02:27,268 DEBUG _trace.py:85  receive_response_headers.complete return_value=(b'HTTP/1.1', 200